# Chapter 7: Data Cleaning and Preparation

- Phần lớn (≈80%) thời gian phân tích dữ liệu là để chuẩn bị: tải, làm sạch, biến đổi, sắp xếp.

- Dữ liệu gốc thường không ở đúng định dạng, cần xử lý lại bằng ngôn ngữ lập trình hoặc công cụ dòng lệnh.

- `Pandas + Python` cung cấp bộ công cụ nhanh, linh hoạt để thao tác dữ liệu.

- `Pandas` được phát triển dựa trên nhu cầu thực tế, người dùng có thể đóng góp ý tưởng.

- Chương này: xử lý dữ liệu thiếu, trùng, chuỗi, biến đổi khác.

- Chương sau: kết hợp và sắp xếp dữ liệu.

## 7.1 Handling Missing Data

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 25
pd.options.display.max_columns = 20
pd.options.display.max_colwidth = 82
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

- Dữ liệu thiếu rất phổ biến trong phân tích dữ liệu.
- Pandas hỗ trợ xử lý dữ liệu thiếu dễ dàng, các thống kê mô tả tự động bỏ qua giá trị thiếu.
- Trong dữ liệu kiểu float64, pandas dùng `NaN` (Not a Number) để biểu diễn giá trị thiếu.
- `NaN` được xem như một sentinel value (giá trị đặc biệt báo hiệu dữ liệu bị thiếu).

> Ví dụ:

In [2]:
import numpy as np
import pandas as pd

In [3]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

- Hàm `isna()` trong pandas trả về một Series kiểu Boolean, với True ở những vị trí có giá trị thiếu (NaN).

In [4]:
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

- Pandas gọi dữ liệu thiếu là `NA`.
- NA có thể do không tồn tại hoặc không quan sát được.
- Cần phân tích dữ liệu thiếu để tìm lỗi hoặc thiên lệch.
- Python None cũng được xem là `NA`.

In [5]:
string_data = pd.Series(["aardvark", np.nan, None, "avocado"])
string_data
string_data.isna()
float_data = pd.Series([1, 2, None], dtype='float64')
float_data
float_data.isna()

0    False
1    False
2     True
dtype: bool

### Filtering Out Missing Data

- Có nhiều cách lọc dữ liệu thiếu.
- Dùng `isna` + indexing thủ công hoặc dropna để nhanh hơn.
- Với Series, dropna giữ lại giá trị **không thiếu** và index.

In [6]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

> Điều này cũng giống như việc làm:

In [7]:
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

- DataFrame có nhiều cách xóa dữ liệu thiếu.
- Có thể xóa hàng/cột toàn NA hoặc có chứa NA.
- `dropna` mặc định xóa hàng có NA.

In [8]:
data = pd.DataFrame([[1., 6.5, 3.], [1., np.nan, np.nan],
                     [np.nan, np.nan, np.nan], [np.nan, 6.5, 3.]])
data
data.dropna()

,0,1,2
0,1.0,6.5,3.0


> Truyền tham số `how="all"` sẽ chỉ xóa những hàng mà tất cả các giá trị đều là NA:

In [9]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


- Các hàm này trả về đối tượng mới, không đổi dữ liệu gốc.
- Xóa cột thì dùng axis="columns".

In [10]:
data[4] = np.nan
data
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


- Muốn giữ lại các hàng chỉ có tối đa một số lượng NA nhất định, dùng tham số `thresh`.

In [12]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df.iloc[:4, 1] = np.nan
df.iloc[:2, 2] = np.nan
df
df.dropna()
df.dropna(thresh=2)

,0,1,2
2,0.464197,NaN,-3.008899
3,0.665883,NaN,-0.691158
4,0.677117,0.940921,-0.326289
5,-1.245130,0.339925,-0.073357
6,-2.467532,0.310566,-0.071165


### Filling In Missing Data

- Thay vì loại bỏ dữ liệu thiếu, có thể điền giá trị thay thế vào chỗ trống.
- Hàm `fillna` thường được dùng để làm việc này.
- Gọi `fillna` với một hằng số sẽ thay thế mọi giá trị thiếu bằng hằng số đó.

In [13]:
df.fillna(0)

,0,1,2
0,0.971391,0.000000,0.000000
1,-1.983681,0.000000,0.000000
2,0.464197,0.000000,-3.008899
3,0.665883,0.000000,-0.691158
4,0.677117,0.940921,-0.326289
5,-1.245130,0.339925,-0.073357
6,-2.467532,0.310566,-0.071165


- Gọi `fillna` với một **dictionary**, bạn có thể chỉ định giá trị thay thế khác nhau cho từng cột.

In [14]:
df.fillna({1: 0.5, 2: 0})

,0,1,2
0,0.971391,0.500000,0.000000
1,-1.983681,0.500000,0.000000
2,0.464197,0.500000,-3.008899
3,0.665883,0.500000,-0.691158
4,0.677117,0.940921,-0.326289
5,-1.245130,0.339925,-0.073357
6,-2.467532,0.310566,-0.071165


Các phương pháp nội suy (interpolation) có sẵn cho **reindexing** cũng có thể được sử dụng với `fillna`.

In [15]:
df = pd.DataFrame(np.random.standard_normal((6, 3)))
df.iloc[2:, 1] = np.nan
df.iloc[4:, 2] = np.nan
df
df.fillna(method="ffill")
df.fillna(method="ffill", limit=2)

/tmp/ipykernel_2690/1607676785.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill")
/tmp/ipykernel_2690/1607676785.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method="ffill", limit=2)


,0,1,2
0,-0.182801,-0.068617,-0.274517
1,1.552538,-1.170861,-0.762375
2,-0.403732,-1.170861,1.637736
3,-0.938865,-1.170861,-1.236091
4,0.724636,NaN,-1.236091
5,0.943089,NaN,-1.236091


- Với `fillna`, có thể thay giá trị thiếu bằng các thống kê đơn giản như **trung vị (median)** hoặc **trung bình (mean)**.

In [16]:
data = pd.Series([1., np.nan, 3.5, np.nan, 7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

## 7.2 Data Transformation

- Chương này đến giờ tập trung vào xử lý dữ liệu thiếu.
- Ngoài ra, còn có các thao tác quan trọng khác như lọc, làm sạch và biến đổi dữ liệu.

### Removing Duplicates

- Trong DataFrame có thể xuất hiện các hàng trùng lặp vì nhiều lý do khác nhau.

> Ví dụ như sau:

In [17]:
data = pd.DataFrame({"k1": ["one", "two"] * 3 + ["two"],
                     "k2": [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


- `duplicated` trả về Series Boolean, xác định hàng nào bị trùng với hàng trước đó.

In [18]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

- `drop_duplicates` trả về DataFrame đã loại bỏ các hàng trùng lặp.

In [19]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


- Mặc định, cả hai phương thức xem xét tất cả các cột.
- Có thể chọn một số cột (ví dụ: chỉ cột "k1") để phát hiện và lọc trùng lặp.

In [20]:
data["v1"] = range(7)
data
data.drop_duplicates(subset=["k1"])

,k1,k2,v1
0,one,1,0
1,two,1,1


- Mặc định, `duplicated` và `drop_duplicates` giữ giá trị trùng đầu tiên.
- Dùng `keep="last"` để giữ giá trị trùng cuối cùng.

In [21]:
data.drop_duplicates(["k1", "k2"], keep="last")

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### Transforming Data Using a Function or Mapping

- Thường cần **biến đổi dữ liệu** dựa trên giá trị trong mảng, Series hoặc cột của DataFrame.
- Ví dụ minh họa: dữ liệu giả định về các loại thịt.

In [22]:
data = pd.DataFrame({"food": ["bacon", "pulled pork", "bacon",
                              "pastrami", "corned beef", "bacon",
                              "pastrami", "honey ham", "nova lox"],
                     "ounces": [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,pastrami,6.0
4,corned beef,7.5
5,bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


- Có thể thêm một cột mới để chỉ rõ loại động vật mà mỗi loại thịt xuất phát từ đó, bằng cách tạo bảng ánh xạ (mapping) giữa thịt và động vật.

In [23]:
meat_to_animal = {
  "bacon": "pig",
  "pulled pork": "pig",
  "pastrami": "cow",
  "corned beef": "cow",
  "honey ham": "pig",
  "nova lox": "salmon"
}

- Phương thức map trên Series cho phép **biến đổi giá trị** bằng cách truyền vào một hàm hoặc một **đối tượng dạng** dictionary để ánh xạ.

In [24]:
data["animal"] = data["food"].map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,pastrami,6.0,cow
4,corned beef,7.5,cow
5,bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


- Ta cũng có thể truyền trực tiếp một hàm để thực hiện toàn bộ việc biến đổi giá trị.

In [25]:
def get_animal(x):
    return meat_to_animal[x]
data["food"].map(get_animal)

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

- `map` là cách tiện lợi để thực hiện biến đổi từng phần tử và các thao tác làm sạch dữ liệu.

### Replacing Values

- `fillna` chỉ là trường hợp đặc biệt của việc thay thế giá trị.
- `map` có thể sửa một phần giá trị, nhưng `replace` đơn giản và linh hoạt hơn.

>  Ví dụ minh họa với một Series:

In [26]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

- Giá trị -999 có thể là dấu hiệu cho dữ liệu thiếu.
- Dùng `replace` để thay bằng NA mà pandas nhận diện, tạo ra Series mới.

In [27]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

- Để thay thế từng giá trị khác nhau, truyền một danh sách các giá trị thay thế.

In [28]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

- Muốn thay mỗi giá trị bằng một giá trị riêng biệt, truyền vào danh sách các giá trị thay thế.

In [28]:
data.replace([-999, -1000], [np.nan, 0])

- Tham số truyền vào cũng có thể là một **dictionary** để chỉ định giá trị thay thế cho từng mục cụ thể.

In [29]:
data.replace({-999: np.nan, -1000: 0})

### Renaming Axis Indexes

- Tương tự như giá trị trong Series, **nhãn trục (axis labels)** cũng có thể được biến đổi bằng hàm hoặc mapping.
- Có thể **sửa trực tiếp nhãn trục** mà không tạo đối tượng mới.

In [29]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                    index=["Ohio", "Colorado", "New York"],
                    columns=["one", "two", "three", "four"])

- Giống Series, **index của trục (axis)** cũng có phương thức `map`.

In [30]:
def transform(x):
    return x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

- Có thể gán trực tiếp cho thuộc tính index để sửa DataFrame ngay tại chỗ (in place):

In [31]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


- Nếu muốn tạo một phiên bản **biến đổi của dataset mà không thay đổi bản gốc**, có thể dùng phương thức `rename`.

In [32]:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


- Đặc biệt, `rename` có thể dùng với **dictionary**, chỉ định nhãn mới cho một tập con các nhãn trục.

In [33]:
data.rename(index={"OHIO": "INDIANA"},
            columns={"three": "peekaboo"})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


- `rename` giúp bạn tránh phải sao chép DataFrame và gán nhãn mới thủ công cho **index và columns**.

### Discretization and Binning

- Dữ liệu liên tục thường được chia thành các “bins” để phân tích.

> Ví dụ: nhóm người theo khoảng tuổi rời rạc.

In [34]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

- Chia dữ liệu thành các khoảng tuổi: 18–25, 26–35, 36–60, 61 trở lên.
- Dùng pandas.cut để thực hiện việc phân nhóm này.

In [35]:
bins = [18, 25, 35, 60, 100]
age_categories = pd.cut(ages, bins)
age_categories

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

- Kết quả từ `pandas.cut` là một **đối tượng Categorical** đặc biệt.
- Mỗi bin được đại diện bởi một interval chứa giới hạn dưới và trên của bin.

In [36]:
age_categories.codes
age_categories.categories
age_categories.categories[0]
pd.value_counts(age_categories)

/tmp/ipykernel_2690/4232757300.py:4: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(age_categories)


(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
Name: count, dtype: int64

- `pd.value_counts(categories)` cho số lượng phần tử trong mỗi bin của `pandas.cut`.
- Trong biểu diễn chuỗi của interval:
    - `( hoặc )` → mở (exclusive)
    - `[ hoặc ]` → đóng (inclusive)

- Có thể thay đổi phía đóng bằng `right=False`.

In [37]:
pd.cut(ages, bins, right=False)

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64, left]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

- Có thể thay nhãn mặc định của các bin bằng cách truyền một danh sách hoặc mảng vào tùy chọn labels.

In [38]:
group_names = ["Youth", "YoungAdult", "MiddleAged", "Senior"]
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

- Nếu truyền một số nguyên cho `pandas.cut` thay vì các giới hạn bin cụ thể, pandas sẽ tạo các bin có độ dài bằng nhau dựa trên giá trị min và max của dữ liệu.

> Ví dụ: dữ liệu phân phối đều được chia thành 4 phần.

In [39]:
data = np.random.uniform(size=20)
pd.cut(data, 4, precision=2)

[(0.041, 0.27], (0.041, 0.27], (0.041, 0.27], (0.041, 0.27], (0.041, 0.27], ..., (0.73, 0.96], (0.73, 0.96], (0.73, 0.96], (0.041, 0.27], (0.27, 0.5]]
Length: 20
Categories (4, interval[float64, right]): [(0.041, 0.27] < (0.27, 0.5] < (0.5, 0.73] < (0.73, 0.96]]

- `precision=2` giới hạn độ chính xác thập phân trong nhãn bin.
- `pandas.qcut` chia dữ liệu dựa trên quantile, tạo các bin có số phần tử xấp xỉ bằng nhau, khác với `pandas.cut` dùng khoảng giá trị.

In [40]:
data = np.random.standard_normal(1000)
quartiles = pd.qcut(data, 4, precision=2)
quartiles
pd.value_counts(quartiles)

/tmp/ipykernel_2690/3037377620.py:4: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(quartiles)


(-3.5, -0.68]      250
(-0.68, -0.037]    250
(-0.037, 0.65]     250
(0.65, 3.33]       250
Name: count, dtype: int64

- Tương tự pandas.cut, bạn có thể truyền quantile tùy chỉnh (giá trị từ 0 đến 1) cho pandas.qcut.

In [41]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.]).value_counts()

(-3.493, -1.322]     100
(-1.322, -0.0374]    400
(-0.0374, 1.26]      400
(1.26, 3.329]        100
Name: count, dtype: int64

### Detecting and Filtering Outliers

- Lọc hoặc biến đổi outlier chủ yếu dựa vào các phép toán trên mảng.

> Ví dụ: DataFrame chứa dữ liệu phân phối chuẩn.

In [42]:
data = pd.DataFrame(np.random.standard_normal((1000, 4)))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.011404,-0.050931,0.011836,0.016183
std,0.998644,0.978273,0.995948,0.976323
min,-3.895986,-3.250802,-3.530634,-2.961295
25%,-0.638668,-0.694035,-0.624531,-0.683791
50%,-0.035263,-0.066937,0.028537,-0.006995
75%,0.701877,0.557371,0.682379,0.704548
max,2.841760,3.066299,3.618159,3.905798


- Giả sử bạn muốn tìm các giá trị trong một cột có giá trị tuyệt đối lớn hơn 3.

In [43]:
col = data[2]
col[col.abs() > 3]

297    3.060544
318    3.238714
657   -3.530634
706    3.618159
Name: 2, dtype: float64

- Để chọn tất cả các hàng có giá trị **>3** hoặc **<-3**, có thể dùng phương thức any trên DataFrame kiểu Boolean.

In [44]:
data[(data.abs() > 3).any(axis="columns")]

,0,1,2,3
41,-3.895986,-1.612891,1.095874,-0.652350
79,0.307969,3.033061,1.275143,0.573837
297,-1.121731,-0.253062,3.060544,-0.439293
318,-0.962667,-0.508862,3.238714,-0.247923
386,-0.409058,-3.250802,-1.509095,-1.067358
485,1.199686,3.064177,0.878100,-0.842616
503,0.048478,3.030815,0.756144,1.850808
657,-0.523260,0.575232,-3.530634,-1.372010
706,0.102574,0.172425,3.618159,0.048229
960,-1.535124,-0.583512,-1.019131,3.905798


- Dùng ngoặc quanh biểu thức so sánh để gọi any.
- Có thể giới hạn (cap) giá trị ngoài một khoảng, ví dụ -3 đến 3.

In [45]:
data[data.abs() > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.012300,-0.050875,0.011449,0.015277
std,0.995532,0.976871,0.991211,0.973125
min,-3.000000,-3.000000,-3.000000,-2.961295
25%,-0.638668,-0.694035,-0.624531,-0.683791
50%,-0.035263,-0.066937,0.028537,-0.006995
75%,0.701877,0.557371,0.682379,0.704548
max,2.841760,3.000000,3.000000,3.000000


- `np.sign(data)` trả về 1 hoặc -1 tùy giá trị trong dữ liệu là dương hay âm.

In [46]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,-1.0,1.0
1,-1.0,1.0,-1.0,-1.0
2,-1.0,-1.0,1.0,1.0
3,-1.0,1.0,-1.0,1.0
4,-1.0,1.0,-1.0,1.0


### Permutation and Random Sampling

- Có thể xáo trộn (permuting) một Series hoặc các hàng trong DataFrame bằng `numpy.random.permutation`.

- Gọi permutation với độ dài trục sẽ trả về một mảng số nguyên xác định thứ tự mới.

In [47]:
df = pd.DataFrame(np.arange(5 * 7).reshape((5, 7)))
df
sampler = np.random.permutation(5)
sampler

array([0, 3, 4, 1, 2])

- Mảng này có thể dùng với iloc hoặc hàm take để truy xuất theo thứ tự mới:

In [48]:
df.take(sampler)
df.iloc[sampler]

,0,1,2,3,4,5,6
0,0,1,2,3,4,5,6
3,21,22,23,24,25,26,27
4,28,29,30,31,32,33,34
1,7,8,9,10,11,12,13
2,14,15,16,17,18,19,20


- Gọi take với **axis="columns"** cũng cho phép chọn một trật tự xáo trộn các cột.

In [49]:
column_sampler = np.random.permutation(7)
column_sampler
df.take(column_sampler, axis="columns")

,4,3,0,6,5,1,2
0,4,3,0,6,5,1,2
1,11,10,7,13,12,8,9
2,18,17,14,20,19,15,16
3,25,24,21,27,26,22,23
4,32,31,28,34,33,29,30


- Để chọn một tập con ngẫu nhiên mà không lặp lại (mỗi hàng chỉ xuất hiện một lần), dùng phương thức `sample` trên Series hoặc DataFrame.

In [50]:
df.sample(n=3)

,0,1,2,3,4,5,6
4,28,29,30,31,32,33,34
3,21,22,23,24,25,26,27
0,0,1,2,3,4,5,6


- Để tạo mẫu có thay thế (cho phép lặp lại các hàng), truyền `replace=True` vào phương thức `sample`.

In [51]:
choices = pd.Series([5, 7, -1, 6, 4])
choices.sample(n=10, replace=True)

2   -1
1    7
1    7
1    7
1    7
4    4
0    5
4    4
4    4
3    6
dtype: int64

### Computing Indicator/Dummy Variables

- Một dạng biến đổi khác cho mô hình thống kê hoặc học máy là **chuyển biến categorical thành ma trận dummy/indicator**.
- Nếu một cột có **k giá trị khác nhau**, tạo DataFrame với **k cột chứa 1 và 0**.
- Pandas có hàm `pandas.get_dummies` để làm việc này, hoặc có thể tự viết hàm.

> Ví dụ minh họa với một DataFrame.

In [52]:
df = pd.DataFrame({"key": ["b", "b", "a", "c", "a", "b"],
                   "data1": range(6)})
df
pd.get_dummies(df["key"], dtype=float)

,a,b,c
0,0.0,1.0,0.0
1,0.0,1.0,0.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,0.0,1.0,0.0


- Trong một số trường hợp, bạn muốn thêm tiền tố (prefix) cho các cột trong DataFrame indicator trước khi gộp với dữ liệu khác.
- Hàm `pandas.get_dummies` có tham số prefix để thực hiện việc này.

In [53]:
dummies = pd.get_dummies(df["key"], prefix="key", dtype=float)
df_with_dummy = df[["data1"]].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0.0,1.0,0.0
1,1,0.0,1.0,0.0
2,2,1.0,0.0,0.0
3,3,0.0,0.0,1.0
4,4,1.0,0.0,0.0
5,5,0.0,1.0,0.0


- Phương thức DataFrame.join sẽ được giải thích chi tiết ở chương sau.
- Nếu một hàng thuộc nhiều category, cần dùng cách khác để tạo biến dummy.

> Ví dụ: bộ dữ liệu MovieLens 1M, được phân tích chi tiết ở Chương 13.

In [55]:
mnames = ["movie_id", "title", "genres"]
movies = pd.read_table("datasets/movielens/movies.dat", sep="::",
                       header=None, names=mnames, engine="python")
movies[:10]

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


- Pandas có phương thức đặc biệt Series.str.get_dummies (các phương thức bắt đầu bằng str. sẽ được giải thích chi tiết ở mục 7.4 “String Manipulation”).
- Phương thức này xử lý trường hợp một hàng thuộc nhiều nhóm, được mã hóa dưới dạng chuỗi phân tách.

In [56]:
dummies = movies["genres"].str.get_dummies("|")
dummies.iloc[:10, :6]

,Action,Adventure,Animation,Children's,Comedy,Crime
0,0,0,1,1,1,0
1,0,1,0,1,0,0
2,0,0,0,0,1,0
3,0,0,0,0,1,0
4,0,0,0,0,1,0
5,1,0,0,0,0,1
6,0,0,0,0,1,0
7,0,1,0,1,0,0
8,1,0,0,0,0,0
9,1,1,0,0,0,0


- Sau đó, giống như trước, bạn có thể gộp với DataFrame movies và thêm tiền tố `"Genre_"` vào tên cột trong DataFrame dummy bằng `add_prefix`.

In [57]:
movies_windic = movies.join(dummies.add_prefix("Genre_"))
movies_windic.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Action                                   0
Genre_Adventure                                0
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Crime                                    0
Genre_Documentary                              0
Genre_Drama                                    0
Genre_Fantasy                                  0
Genre_Film-Noir                                0
Genre_Horror                                   0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Romance                                  0
Genre_Sci-Fi                                   0
Genre_Thriller                                 0
Genre_War                                      0
Genre_Western       

- Một cách hữu ích trong thống kê là kết hợp `pandas.get_dummies` với hàm **discretization** như `pandas.cut`.

In [58]:
np.random.seed(12345) # to make the example repeatable
values = np.random.uniform(size=10)
values
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,False,False,False,False,True
1,False,True,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,False,False,True,False,False
5,False,False,True,False,False
6,False,False,False,False,True
7,False,False,False,True,False
8,False,False,False,True,False
9,False,False,False,True,False


## 7.3 Extension Data Types

- Pandas được xây dựng dựa trên NumPy, thư viện xử lý mảng số, và nhiều khái niệm như dữ liệu thiếu dùng khả năng có sẵn trong NumPy.
- Việc dựa vào NumPy cũng dẫn đến một số hạn chế:
- Xử lý dữ liệu thiếu cho kiểu integer và Boolean không hoàn chỉnh → pandas chuyển sang float64 và dùng np.nan, gây ra một số vấn đề nhỏ trong thuật toán.
- Dữ liệu nhiều chuỗi tốn nhiều bộ nhớ và thời gian tính toán.
- Một số kiểu dữ liệu như time intervals, timedeltas, timestamps với timezone khó xử lý hiệu quả mà không dùng mảng Python tốn kém.
- Gần đây, pandas phát triển hệ thống kiểu mở rộng cho phép thêm kiểu dữ liệu mới, có thể sử dụng cùng với dữ liệu từ NumPy.
> Ví dụ: tạo một Series kiểu integer có giá trị thiếu.

In [59]:
s = pd.Series([1, 2, 3, None])
s
s.dtype

dtype('float64')

- Chủ yếu để tương thích ngược, Series vẫn dùng float64 và np.nan cho giá trị thiếu.
- Có thể tạo Series với kiểu dữ liệu mới sử dụng pandas.Int64Dtype.

In [60]:
s = pd.Series([1, 2, 3, None], dtype=pd.Int64Dtype())
s
s.isna()
s.dtype

Int64Dtype()

- Kết quả **<NA>** biểu thị giá trị bị thiếu trong extension type array.
> Sử dụng giá trị đặc biệt pandas.NA.

In [61]:
s[3]
s[3] is pd.NA

True

- Có thể dùng "Int64" thay cho **pd.Int64Dtype()** để chỉ kiểu dữ liệu.
- Chú ý viết hoa, nếu không sẽ là kiểu NumPy không phải extension type.

In [62]:
s = pd.Series([1, 2, 3, None], dtype="Int64")

- Pandas cũng có extension type chuyên dụng cho dữ liệu chuỗi, không dùng mảng object của NumPy.
- Kiểu này yêu cầu thư viện pyarrow, có thể cần cài đặt riêng.

In [63]:
s = pd.Series(['one', 'two', None, 'three'], dtype=pd.StringDtype())
s

- Các string arrays này thường tiết kiệm bộ nhớ và tính toán nhanh hơn trên dữ liệu lớn.
- Một extension type quan trọng khác là Categorical (xem chi tiết ở mục 7.5).
- Danh sách extension type đầy đủ xem Bảng 7-3.
- Có thể dùng phương thức astype của Series để chuyển đổi sang các extension type trong quá trình làm sạch dữ liệu.

In [63]:
df = pd.DataFrame({"A": [1, 2, None, 4],
                   "B": ["one", "two", "three", None],
                   "C": [False, None, False, True]})
df
df["A"] = df["A"].astype("Int64")
df["B"] = df["B"].astype("string")
df["C"] = df["C"].astype("boolean")
df

,A,B,C
0,1,one,False
1,2,two,<NA>
2,<NA>,three,False
3,4,<NA>,True


## 7.4 String Manipulation

- Python từ lâu đã phổ biến cho xử lý dữ liệu thô, đặc biệt là chuỗi và văn bản, nhờ các phương thức sẵn có của string.
- Với các thao tác phức tạp hơn cần regular expressions.
- Pandas giúp áp dụng chuỗi và regex trên toàn bộ mảng dữ liệu một cách gọn gàng, đồng thời xử lý dữ liệu thiếu.

### Python Built-In String Object Methods

- Trong nhiều ứng dụng xử lý chuỗi, các phương thức string sẵn có là đủ.

> Ví dụ: chuỗi phân tách bằng dấu phẩy có thể tách thành các phần bằng `split`.

In [64]:
val = "a,b,  guido"
val.split(",")

['a', 'b', '  guido']

- `split` thường được kết hợp với `strip` để loại bỏ dấu cách thừa (bao gồm xuống dòng).

In [65]:
pieces = [x.strip() for x in val.split(",")]
pieces

['a', 'b', 'guido']

- Các **substring** này có thể được nối lại với delimiter `::` bằng phép cộng `(+)`.

In [66]:
first, second, third = pieces
first + "::" + second + "::" + third

'a::b::guido'

- Tuy nhiên, cách này không phù hợp tổng quát.
- Cách nhanh hơn và Pythonic là truyền danh sách hoặc tuple vào phương thức `"::".join()`.

In [67]:
"::".join(pieces)

'a::b::guido'

- Các phương thức khác liên quan đến tìm vị trí substring.
- Dùng in là cách tốt nhất để kiểm tra substring, ngoài ra còn có `index` và `find`.

In [68]:
"guido" in val
val.index(",")
val.find(":")

-1

> Lưu ý: `index` sẽ báo lỗi nếu không tìm thấy chuỗi, còn `find` trả về -1.

In [69]:
val.index(":")

ValueError: substring not found

- Ngoài ra, count trả về số lần xuất hiện của một substring cụ thể.

In [70]:
val.count(",")

2

- `replace` thay thế các mẫu (pattern) bằng mẫu khác.
- Cũng thường dùng để xóa pattern bằng cách truyền chuỗi rỗng.

In [71]:
val.replace(",", "::")
val.replace(",", "")

'ab  guido'

### Regular Expressions

- Regular expressions (regex) cung cấp cách linh hoạt để tìm kiếm hoặc khớp các mẫu chuỗi, thường là phức tạp.
- Python dùng module re để áp dụng regex cho chuỗi.
- Các hàm trong re thuộc ba nhóm: pattern matching, substitution, và splitting.

> Ví dụ: muốn tách chuỗi theo số lượng khoảng trắng biến thiên (tabs, spaces, newlines), dùng regex \s+.

In [72]:
import re
text = "foo    bar\t baz  \tqux"
re.split(r"\s+", text)

['foo', 'bar', 'baz', 'qux']

- Khi gọi `re.split(r"\s+", text)`, regex sẽ được biên dịch trước, sau đó dùng phương thức split trên chuỗi.
- Có thể tự biên dịch regex bằng `re.compile` để tạo regex object có thể tái sử dụng.

In [73]:
regex = re.compile(r"\s+")
regex.split(text)

['foo', 'bar', 'baz', 'qux']

- Nếu muốn lấy danh sách tất cả các mẫu khớp với regex, dùng phương thức `findall`.

In [74]:
regex.findall(text)

['    ', '\t ', '  \t']

- Nên dùng `re.compile` khi áp dụng cùng một regex cho nhiều chuỗi để tiết kiệm CPU.
- `match` và `search` liên quan đến `findall`:
    - `findall` trả về tất cả khớp.
    - `search` trả về khớp đầu tiên.
    - `match` chỉ khớp tại đầu chuỗi.

> Ví dụ: regex để nhận diện email trong một đoạn văn bản.

In [75]:
text = """Thuong thuong@google.com
Thi thi@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com"""
pattern = r"[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}"

# re.IGNORECASE makes the regex case insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

- Dùng `findall` trên văn bản sẽ trả về danh sách các địa chỉ email:

In [76]:
regex.findall(text)

['thuong@google.com', 'thi@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

- `search` trả về một match object đặc biệt cho email đầu tiên trong văn bản.
- Match object chỉ cho biết vị trí bắt đầu và kết thúc của pattern trong chuỗi.

In [77]:
m = regex.search(text)
m
text[m.start():m.end()]

'thuong@google.com'

- `regex.match` trả về None, vì nó chỉ khớp nếu pattern xuất hiện tại đầu chuỗi.

In [78]:
print(regex.match(text))

None


- Tương tự, sub trả về một chuỗi mới với các pattern được thay bằng chuỗi khác.

In [79]:
print(regex.sub("REDACTED", text))

Thuong REDACTED
Thi REDACTED
Rob REDACTED
Ryan REDACTED


- Giả sử muốn tìm email và chia mỗi email thành ba phần: username, domain name, domain suffix.
- Cách làm: đặt ngoặc quanh các phần trong pattern để tách nhóm.

In [80]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
regex = re.compile(pattern, flags=re.IGNORECASE)

- Match object từ regex đã chỉnh sửa sẽ trả về một tuple các thành phần của pattern bằng phương thức `groups`.

In [81]:
m = regex.match("wesm@bright.net")
m.groups()

('wesm', 'bright', 'net')

- `findall` trả về một danh sách các tuple nếu pattern có các nhóm (groups).

In [82]:
regex.findall(text)

[('thuong', 'google', 'com'),
 ('thi', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

- `sub` cũng có thể truy cập các group trong mỗi match bằng các ký hiệu đặc biệt như `\1`, `\2`:
    - `\1` → nhóm đầu tiên
    - `\2` → nhóm thứ hai
    - v.v.

In [83]:
print(regex.sub(r"Username: \1, Domain: \2, Suffix: \3", text))

Thuong Username: thuong, Domain: google, Suffix: com
Thi Username: thi, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com


### String Functions in pandas

- Làm sạch một dataset lộn xộn thường yêu cầu nhiều xử lý chuỗi.
- Một khó khăn nữa là cột chuỗi đôi khi có dữ liệu thiếu.

In [84]:
data = {"Dave": "dave@google.com", "Steve": "steve@gmail.com",
        "Rob": "rob@gmail.com", "Wes": np.nan}
data = pd.Series(data)
data
data.isna()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

- Có thể áp dụng các phương thức string và regex cho từng giá trị bằng data.map, nhưng sẽ lỗi nếu gặp NA.
- Để xử lý, Series có các phương thức array-oriented qua thuộc tính str, bỏ qua và giữ nguyên NA.

> Ví dụ: kiểm tra xem email có chứa "gmail" bằng str.contains.

In [85]:
data.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

- Kết quả của phép toán này có kiểu object dtype.
- Pandas cung cấp các extension type cho string, integer, Boolean, giúp xử lý dữ liệu thiếu tốt hơn so với trước đây.

In [86]:
data_as_string_ext = data.astype('string')
data_as_string_ext
data_as_string_ext.str.contains("gmail")

Dave     False
Steve     True
Rob       True
Wes       <NA>
dtype: boolean

- Regular expressions cũng có thể dùng, kết hợp với các tùy chọn của re, ví dụ IGNORECASE.

In [87]:
pattern = r"([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})"
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

- Có một vài cách để lấy phần tử theo vector hóa:
    - Dùng `str.get`
    - Hoặc index trực tiếp qua thuộc tính `str`.

In [88]:
matches = data.str.findall(pattern, flags=re.IGNORECASE).str[0]
matches
matches.str.get(1)

Dave     google
Steve     gmail
Rob       gmail
Wes         NaN
dtype: object

- Tương tự, có thể cắt (slice) chuỗi bằng cú pháp này.

In [89]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

- str.extract trả về các group đã capture của regex dưới dạng một DataFrame.

In [90]:
data.str.extract(pattern, flags=re.IGNORECASE)

,0,1,2
Dave,dave,google,com
Steve,steve,gmail,com
Rob,rob,gmail,com
Wes,NaN,NaN,NaN


## 7.5 Categorical Data

- Phần này giới thiệu pandas Categorical type.
- Loại này giúp cải thiện hiệu suất và tiết kiệm bộ nhớ trong một số thao tác pandas.
- Đồng thời cung cấp các công cụ hỗ trợ sử dụng dữ liệu categorical trong thống kê và học máy.

### Background and Motivation

- Thường thì một cột có nhiều giá trị lặp lại từ một tập giá trị nhỏ.
- Các hàm `unique` và `value_counts` giúp:
    - `unique`: lấy các giá trị khác nhau
    - `value_counts`: đếm tần suất xuất hiện của mỗi giá trị.

In [91]:
values = pd.Series(['apple', 'orange', 'apple',
                    'apple'] * 2)
values
pd.unique(values)
pd.value_counts(values)

/tmp/ipykernel_2690/3503291848.py:5: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(values)


apple     6
orange    2
Name: count, dtype: int64

- Nhiều hệ thống dữ liệu (data warehouse, thống kê,…) đã phát triển các cách đặc biệt để lưu dữ liệu có giá trị lặp lại, giúp tiết kiệm bộ nhớ và tăng hiệu quả tính toán.
- Trong data warehousing, best practice là dùng dimension tables:
    - Chứa các giá trị khác nhau
    - Các quan sát chính lưu dưới dạng integer key tham chiếu tới dimension table.

In [92]:
values = pd.Series([0, 1, 0, 0] * 2)
dim = pd.Series(['apple', 'orange'])
values
dim

0     apple
1    orange
dtype: object

- Có thể dùng `take` để khôi phục Series ban đầu từ các key integer.

In [93]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

### Categorical Extension Type in pandas

- Pandas có extension type Categorical dùng integer-based encoding.
- Đây là kỹ thuật nén dữ liệu phổ biến cho dữ liệu có nhiều giá trị lặp lại, giúp tăng tốc độ và tiết kiệm bộ nhớ, đặc biệt với dữ liệu chuỗi.

> Ví dụ: Series đã xem trước đó.

In [94]:
fruits = ['apple', 'orange', 'apple', 'apple'] * 2
N = len(fruits)
rng = np.random.default_rng(seed=12345)
df = pd.DataFrame({'fruit': fruits,
                   'basket_id': np.arange(N),
                   'count': rng.integers(3, 15, size=N),
                   'weight': rng.uniform(0, 4, size=N)},
                  columns=['basket_id', 'fruit', 'count', 'weight'])
df

,basket_id,fruit,count,weight
0,0,apple,11,1.564438
1,1,orange,5,1.331256
2,2,apple,12,2.393235
3,3,apple,6,0.746937
4,4,apple,5,2.691024
5,5,orange,12,3.767211
6,6,apple,10,0.992983
7,7,apple,11,3.795525


- Ở đây, `df['fruit']` là mảng các string Python.
- Có thể chuyển sang categorical bằng cách gọi:

In [95]:
fruit_cat = df['fruit'].astype('category')
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

- Giá trị của `fruit_cat` giờ là `pandas.Categorical`, có thể truy cập qua thuộc tính `.array`.

In [96]:
c = fruit_cat.array
type(c)

pandas.core.arrays.categorical.Categorical

- Categorical object có hai thuộc tính: `categories` và `codes`.

In [97]:
c.categories
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

> Một mẹo hữu ích để lấy mapping giữa codes và categories là:

In [98]:
dict(enumerate(c.categories))

{0: 'apple', 1: 'orange'}

- Có thể chuyển cột DataFrame sang categorical bằng cách gán kết quả đã chuyển đổi.

In [99]:
df['fruit'] = df['fruit'].astype('category')
df["fruit"]

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

- Cũng có thể tạo **pandas.Categorical** trực tiếp từ các sequence khác của Python.

In [100]:
my_categories = pd.Categorical(['foo', 'bar', 'baz', 'foo', 'bar'])
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

- Nếu đã có dữ liệu categorical được mã hóa từ nguồn khác, có thể dùng `from_codes` constructor.

In [101]:
categories = ['foo', 'bar', 'baz']
codes = [0, 1, 2, 0, 0, 1]
my_cats_2 = pd.Categorical.from_codes(codes, categories)
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

- Nếu không chỉ định, các categorical conversion không giả định thứ tự cụ thể của categories.
- Mảng categories có thể khác nhau tùy dữ liệu đầu vào.
- Khi dùng `from_codes` hoặc các constructor khác, có thể chỉ định thứ tự có ý nghĩa cho categories.

In [102]:
ordered_cat = pd.Categorical.from_codes(codes, categories,
                                        ordered=True)
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

- Kết quả `[foo < bar < baz]` nghĩa là `'foo'` đứng trước `'bar'`,…
- Một categorical không có thứ tự có thể được chuyển thành ordered bằng as_ordered.

In [103]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

### Computations with Categoricals

- Dùng Categorical trong pandas so với phiên bản không mã hóa (mảng string) về cơ bản giống nhau.
- Một số phần của pandas, như groupby, chạy nhanh hơn với categorical.
- Một số hàm khác có thể dùng ordered flag.

> Ví dụ: dùng `pandas.qcut` để bin dữ liệu số ngẫu nhiên; hàm này trả về `pandas.Categorical`.

In [104]:
rng = np.random.default_rng(seed=12345)
draws = rng.standard_normal(1000)
draws[:5]

array([-1.42382504,  1.26372846, -0.87066174, -0.25917323, -0.07534331])

- Thực hiện chia dữ liệu thành các quartile và trích xuất thống kê.

In [105]:
bins = pd.qcut(draws, 4)
bins

[(-3.121, -0.675], (0.687, 3.211], (-3.121, -0.675], (-0.675, 0.0134], (-0.675, 0.0134], ..., (0.0134, 0.687], (0.0134, 0.687], (-0.675, 0.0134], (0.0134, 0.687], (-0.675, 0.0134]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.121, -0.675] < (-0.675, 0.0134] < (0.0134, 0.687] < (0.687, 3.211]]

- Mặc dù hữu ích, quartile thực tế có thể ít trực quan khi báo cáo.
- Có thể dùng labels trong qcut để đặt tên cho các quartile.

In [106]:
bins = pd.qcut(draws, 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
bins
bins.codes[:10]

array([0, 3, 0, 1, 1, 0, 0, 2, 2, 0], dtype=int8)

- Categorical với nhãn không chứa thông tin về biên của bin trong dữ liệu.
- Có thể dùng `groupby` để trích xuất thống kê tóm tắt.

In [107]:
bins = pd.Series(bins, name='quartile')
results = (pd.Series(draws)
           .groupby(bins)
           .agg(['count', 'min', 'max'])
           .reset_index())
results

/tmp/ipykernel_2690/2483392743.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(bins)


,quartile,count,min,max
0,Q1,250,-3.119609,-0.678494
1,Q2,250,-0.673305,0.008009
2,Q3,250,0.018753,0.686183
3,Q4,250,0.688282,3.211418


- Cột 'quartile' trong kết quả giữ thông tin categorical gốc, bao gồm cả thứ tự, từ bins.

In [108]:
results['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

##### Better performance with categoricals

- Như đã nói, categorical types giúp cải thiện hiệu suất và tiết kiệm bộ nhớ.

> Ví dụ: Series với 10 triệu phần tử nhưng số lượng category nhỏ.

In [109]:
N = 10_000_000
labels = pd.Series(['foo', 'bar', 'baz', 'qux'] * (N // 4))

- Bây giờ chuyển labels sang categorical.

In [110]:
categories = labels.astype('category')

- Lưu ý rằng labels sử dụng bộ nhớ nhiều hơn so với categories.

In [111]:
labels.memory_usage(deep=True)
categories.memory_usage(deep=True)

10000512

- Việc chuyển sang category không miễn phí, nhưng chỉ là chi phí một lần.

In [112]:
%time _ = labels.astype('category')

CPU times: user 380 ms, sys: 46 ms, total: 426 ms
Wall time: 429 ms


- GroupBy có thể nhanh hơn nhiều với categorical vì thuật toán sử dụng mảng codes dạng integer thay vì mảng string.

> Ví dụ: so sánh hiệu suất value_counts(), vốn dùng cơ chế GroupBy bên trong.

In [113]:
%timeit labels.value_counts()
%timeit categories.value_counts()

491 ms ± 30.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
24.4 ms ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Categorical Methods

- Series chứa dữ liệu categorical có nhiều phương thức đặc biệt, tương tự Series.str cho chuỗi.
- Cũng cho phép truy cập tiện lợi tới categories và codes.

> Ví dụ: Series.

In [114]:
s = pd.Series(['a', 'b', 'c', 'd'] * 2)
cat_s = s.astype('category')
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

- Thuộc tính đặc biệt `cat` cho phép truy cập các phương thức categorical.

In [115]:
cat_s.cat.codes
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

- Giả sử biết tập categories thực tế lớn hơn bốn giá trị quan sát trong dữ liệu.
- Có thể dùng `set_categories` để thay đổi chúng.

In [116]:
actual_categories = ['a', 'b', 'c', 'd', 'e']
cat_s2 = cat_s.cat.set_categories(actual_categories)
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

- Dữ liệu có vẻ không thay đổi, nhưng categories mới sẽ ảnh hưởng đến các phép toán sử dụng chúng.

> Ví dụ: `value_counts` tôn trọng các category nếu có.

In [117]:
cat_s.value_counts()
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
Name: count, dtype: int64

- Trong dữ liệu lớn, categorical thường dùng để tiết kiệm bộ nhớ và tăng hiệu suất.
- Sau khi lọc, nhiều category có thể không xuất hiện trong dữ liệu.
- Có thể dùng `remove_unused_categories` để loại bỏ các category không dùng.

In [118]:
cat_s3 = cat_s[cat_s.isin(['a', 'b'])]
cat_s3
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

##### Creating dummy variables for modeling

- Trong thống kê hoặc học máy, dữ liệu categorical thường được chuyển thành dummy variables (one-hot encoding).
- Tạo DataFrame với một cột cho mỗi category; cột chứa 1 nếu giá trị xuất hiện, 0 nếu không.

> Ví dụ: dựa trên ví dụ trước.

In [119]:
cat_s = pd.Series(['a', 'b', 'c', 'd'] * 2, dtype='category')

- Như đã nói,  `pandas.get_dummies` chuyển dữ liệu categorical một chiều thành DataFrame chứa dummy variables.

In [120]:
pd.get_dummies(cat_s, dtype=float)

,a,b,c,d
0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0
6,0.0,0.0,1.0,0.0
7,0.0,0.0,0.0,1.0


## 7.6 Conclusion

- Chuẩn bị dữ liệu hiệu quả giúp tăng năng suất, dành nhiều thời gian phân tích hơn và ít thời gian chuẩn bị dữ liệu.
- Chương này đã giới thiệu nhiều công cụ, nhưng không đầy đủ.
- Chương sau sẽ trình bày về join và group trong pandas.